## Perturbation and sensitivity analysis

Consider a general optimization problem:
$$	\begin{equation*}
	\begin{array}{llll}
	\text{minimize}  & f_0(x) &\\
	\text{subject to}& f_i(x)   & \le 0,  &i=1 ,\dots, m\\
	& h_i(x)   & = 0,  &i=1 ,\dots, p
	\end{array}
	\end{equation*}$$

Now introduce constants $u_i$ and $v_i$ and consider a perturbed problem as follows:

$$	\begin{equation*}
	\begin{array}{llll}
	\text{minimize}  & f_0(x) &\\
	\text{subject to}& f_i(x)   & \le u_i,  &i=1 ,\dots, m\\
	& h_i(x)   & = v_i,  &i=1 ,\dots, p
	\end{array}
	\end{equation*}$$	

Note that $u_i>0$ corresponds to loosening the corresponding constraint, while $u_i<0$ corresponds to tightening it.

Define $p^\star(u,v)$ as the optimal value of the perturbed problem ($p^\star(u,v)=+\infty$ is possible, even if $p^\star(0,0)$ is finite).

### Exercise 1

Assume strong duality holds for the original problem. 
Let $\lambda^\star, \nu^\star$ be the dual optimum solution.
Show that $p^\star(u,v) \ge p^\star(0,0)-{\lambda^\star}^T u -{\nu^\star}^T v$. 

If $p^\star(u,v)=+\infty$ it's true. If $p^\star(u,v)<+\infty$ we have $L_{(u,v)}(x,\lambda,v) = f_0(x) + \sum \lambda_i(f_i(x)-u_i) + \sum \nu_i (h_i(x)-v_i) = L_{(0,0)} - \lambda^{T}u - \nu^{T}v$. 
Therefore $\inf_x L_{(u,v)} = \inf_x L_{(0,0)}- \lambda^{T}u - \nu^{T}v$. So from strong duality $p^\star(u,v) \geq \max g(0,0)- \lambda^{T}u - \nu^{T}v =p^\star(0,0) - \lambda^{T}u - \nu^{T}v$
### Exercise 2

Use the result of Exercise 1 to answer the following questions:
- What happens with the objective function if $\lambda^\star_i>0$ and we tighten the $i$-th constraint ($u_i<0$)? 
- What happens with the objective function if $\lambda^\star_i>0$ and we loosen the $i$-th constraint ($u_i>0$)? 
- What happens with the objective function if $\nu^\star_i<0$ and $v_i>0$? 

In the first case objective function will be bigger, in the second case function can decrease at most $\sum|\lambda^\star_i u_i|$. The third one similar.
### Exercise 3

Assume strong duality holds for the original problem and additionally $p^\star(u,v)$ is differentiable at $u=0$, $v=0$.
Show that
$$\frac{\partial p^\star(0,0)}{\partial u_i} = -\lambda_i^\star, \quad \frac{\partial p^\star(0,0)}{\partial v_i} = -\nu_i^\star.$$

True from exercise 1.

### Exercise 4: Supply and Demand Chain problem (programming)

Consider the graph below. 

![img](graph.jpg)

The graph represents a network for sending goods.
Each node $v$ has associated a required supply and demand balance $b_v$ (supply - demand): if $b(v)>0$, then $v$ represents a producer, and otherwise a consumer.
Each edge $(u,v)$ labelled with $c_{u,v}$ represents the possibility of sending goods from $u$ to $v$, at the cost of $c_{u,v}$ PLN per kg.
Additionally, for each edge we know its capacity which bounds the amount of goods (in kgs) that can be sent through this edge.
The goal is to find the flow of goods of minimum cost.

Let $n=6$ be the number of vertices and $m=9$ be the number of edges.
Let $c\in \mathbb{R}^m$, $b\in \mathbb{R}^n$, and $capacity\in \mathbb{R}^m$ be the corresponding vectors of costs, balances and capacities (specified in the code below).

1. Solve the supply and demand chain problem by representing it as the following LP (for an approprietly constructed matrix $A$), using cvxpy:
$$\begin{equation*}
	\begin{array}{ll}
	\text{minimize}  & c^Tx\\
	\text{subject to}& Ax = b \\
	                 & x \le capacity\\
	                 & x \ge 0\\
	\end{array}
	\end{equation*}$$ 

2. For which edge modifying its capacity value will have the biggest effect on the value of the objective function? Will this value increase or decrease if we decrease the capacity? Answer this question using the claim from Exercise 3. (If ``constraints`` store your list of constraints, ``constraints[i].dual_value`` stores the value of the corresponding dual variable in an optimal dual solution). 


3. Verify your answers experimentally: for $\epsilon=0.1$, for each edge $e$, increase $capacity[e]$ by $\epsilon$, solve the new problem and check how much the objecive changed. 

In [28]:
import cvxpy as cp
import numpy as np

c = [2, 5, 5, 2, 7, 9, 2, 4, 3]
b = [10.0, 1, -2, -3, 4, -10]
capacity = [7, 4, 5, 5, 2, 6, 7, 4, 5]

edges_l = ('AB', 'BC', 'AD', 'BD', 'BE', 'BF', 'CF', 'DE', 'EF')
edges = list(map(lambda s: tuple(map(lambda c: ord(c)-ord('A'), s)), edges_l))
print(f'edges = {edges}')

m = len(c)
n = len(b)

edges = [(0, 1), (1, 2), (0, 3), (1, 3), (1, 4), (1, 5), (2, 5), (3, 4), (4, 5)]


In [29]:
x = cp.Variable(m)

A = np.zeros((n, m))

for i in range(m):
    for j in range(n):
        if j == edges[i][0]:
            A[j][i] = 1
        elif j == edges[i][1]:
            A[j][i] = -1

constraints = [x <= capacity,
               -x <= 0,
               A@x == b]


obj = cp.Minimize(cp.sum(cp.multiply(c, x)))

prob = cp.Problem(obj, constraints)
prob.solve(verbose=True,solver=cp.GUROBI)
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x.value)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Apr 26 09:19:52 AM: Your problem has 9 variables, 3 constraints, and 0 parameters.
(CVXPY) Apr 26 09:19:52 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 26 09:19:52 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 26 09:19:52 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 26 09:19:52 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 26 09:19:52 AM: Compiling problem (target solver=GUROBI).
(CV

In [30]:
# Capacity constraints
constraints[0].dual_value

array([ 1.,  2., -0., -0., -0., -0., -0., -0., -0.])

In [33]:
# 3
optimal_val = []

epsilon = 0.1

for i in range(m):
    if i==0:
        capacity[0] += epsilon
    elif i==m-1:
        capacity[m-2] -= epsilon
        capacity[m-1] += epsilon
    else:
        capacity[i-1] -= epsilon
        capacity[i] += epsilon

    x_i = cp.Variable(m)

    constraints_i = [x_i <= capacity,
                -x_i <= 0,
                A@x_i == b]


    obj_i = cp.Minimize(cp.sum(cp.multiply(c, x_i)))

    prob_i = cp.Problem(obj_i, constraints_i)
    prob_i.solve(verbose=True,solver=cp.GUROBI)
    optimal_val += [i, prob_i.value]
    print(constraints[0].dual_value)
print(optimal_val)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Apr 26 09:20:50 AM: Your problem has 9 variables, 3 constraints, and 0 parameters.
(CVXPY) Apr 26 09:20:50 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 26 09:20:50 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 26 09:20:50 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 26 09:20:50 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 26 09:20:50 AM: Compiling problem (target solver=GUROBI).
(CV

As we can see, the objective has changed, as indicated in exercise 3.